In [109]:
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm

In [122]:
class DiGraph():
    '''
    basic directed graph
    '''
    
    def __init__(self, default_attribute = None):
        """
        internally we save it both as an adjacency list and as a list of edges
        for convenience in computation
        """
        
        def default_None_dict():
            return defaultdict(lambda : default_attribute)
        
        self.edges_list = defaultdict(lambda : default_attribute)
        self.adjacency = defaultdict(default_None_dict)
    
    @property
    def graph_type(self):
        return 'Directed Graph'
    
    def __repr__(self):
        """"""
        return 'DiGraph()'
    
    def __getitem__(self, key):
        return list(self.adjacency[key].keys())
    
    def add_node(self, node):
        self.adjacency[node]
    
    @property
    def nodes(self):
        return list(self.adjacency.keys())
    
    @property
    def edges(self):
        return list(self.edges_list.keys())
    
    def adj(self):
        return(dict(map(lambda x: (x[0], list(x[1].keys())), self.adjacency.items())))
    
    def add_edge(self, edge0, edge1):
        self.add_node(edge1)
        self.adjacency[edge0][edge1]
        self.edges_list[(edge0, edge1,)]
    
    @property
    def n_nodes(self):
        return len(self.adjacency)
    
    @property
    def n_edges(self):
        return len(self.edges_list)
    
    @property
    def get_average_links(self):
        return self.n_edges / self.n_nodes
    
    @property
    def get_density(self):
        possible_edges = self.n_nodes * (self.n_nodes - 1)
        return self.n_edges / possible_edges
    
    @property
    def isDense(self):
        is_dense = self.get_density >= 0.5
        return is_dense
    
    @property
    def isSparse(self):
        return (not self.isDense)
    
    
class weightedDiGraph(DiGraph):
    '''
    weighted directed graph
    '''
    def __init__(self):
        super().__init__(0)
    
    
    def __repr__(self):
        """"""
        return 'weightedDiGraph()'
    
    @property
    def graph_type(self):
        return 'Weighted Directed Graph'
    
    def __getitem__(self, key):
        return dict(self.adjacency[key])
    
    @property
    def edges(self):
        return list(self.edges_list.items())
    
    def adj(self):
        return(dict(map(lambda x: (x[0], dict(x[1])), self.adjacency.items())))
    
    def add_edge(self, edge0, edge1, weight):
        self.add_node(edge1)
        self.adjacency[edge0][edge1] += weight
        self.edges_list[(edge0, edge1)] += weight
    
    def union(self, another_weightedDiGraph):
        for edge, weight in another_weightedDiGraph.edges:
            self.add_edge(*edge, weight)

        
def search_timestamp(edges_list, time, mode):
    
    if mode == 'start':
        if edges_list[0][2] >= time:
            return(0)
        elif edges_list[-1][2] < time:
            raise ValueError('Start time too high in search_timestamp()')
    elif mode == 'end':
        if edges_list[0][2] > time:
            raise ValueError('End time too low in search_timestamp()')
        elif edges_list[-1][2] <= time:
            return(len(edges_list))
    else:
        raise ValueError('Third argument in search_timestamp() must be either "start" or "end"')
    
    
    start_idx = 0
    end_idx = len(edges_list)
    
    while (end_idx - start_idx) > 1:
        
        middle_idx = (end_idx + start_idx)//2
        
        if edges_list[middle_idx][2] < time:
            start_idx = middle_idx
            
        elif edges_list[middle_idx][2] > time:
            end_idx = middle_idx
            
        else:
            
            if mode == 'start':
                while (edges_list[middle_idx][2] == time):
                    middle_idx -= 1
                start_idx = middle_idx + 1
                return(start_idx)
            else:
                while (edges_list[middle_idx][2] == time):
                    middle_idx += 1
                start_idx = middle_idx
                return(start_idx)
    
    start_idx+=1
    
    return(start_idx)

    
class MultiDiGraph(DiGraph):
    
    def __init__(self):
        super().__init__()
    
    def __repr__(self):
        """"""
        return 'DiMultiGraph()'
    
    @property
    def graph_type(self):
        return 'Directed MultiGraph'
    
    def add_edge(self, edge0, edge1, timestamp):
        self.adjacency[edge1]
        self.adjacency[edge0][(edge1, timestamp)]
        self.edges_list[(edge0, edge1, timestamp)]
    
    def get_graph(self, filename):
        """
        builds the graph from an input file
        """
        
        with open(filename, 'r') as file:
            for row in tqdm(file, total = 26000000):
                row = row.strip().split()
                start_node, end_node, timestamp = list(map(int, row))
                timestamp = timestamp # we should round the timestamps HERE###############################################################
                self.add_edge(start_node, end_node, timestamp)
    
    
    def get_DiGraph_from_interval(self, start_time = None, end_time = None, weight = 1):
        '''
        builds a digraph that contains only the selected time interval
        '''
        
        if start_time == None:
            start_time = self.edges[0][2]
        if end_time == None:
            end_time = self.edges[-1][2]
        
        final_graph = weightedDiGraph()
        start_edge = search_timestamp(self.edges, start_time, mode = 'start')  # returns the index of the smallest edge
                                                                               # with a timestamp bigger or equal than start_time
        
        end_edge = search_timestamp(self.edges, end_time, mode = 'end')        # returns the index of the smallest edge
                                                                               # with a timestamp smaller than end_time
        
        for edge in self.edges[start_edge : end_edge]:
            final_graph.add_edge(*(edge[0:2]), weight)
        
        return(final_graph)
    

Prova DiGraph()

In [99]:
prova = DiGraph()
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_node(7)
prova.add_node('7')
prova.add_node(42)
prova.add_node('la risposta')
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_edge(7, '7')
prova.add_edge(7, '7')
prova.add_edge('la risposta', 42)
prova.add_edge('la risposta', 'culo')
print('Lati correnti', prova.edges)
print('Nodi correnti', prova.nodes)
print('Lista di adiacenza', prova.adj())
print('Le risposte sono', prova['la risposta'])
print('Questo grafo è un', prova.graph_type)
print('Ci sono', prova.n_nodes, 'nodi e', prova.n_edges, 'lati')
print('Il numero medio di link è', prova.get_average_links)
print(prova)

Lista di adiacenza {}
Nodi correnti []
Lista di adiacenza {7: [], '7': [], 42: [], 'la risposta': []}
Nodi correnti [7, '7', 42, 'la risposta']
Lati correnti [(7, '7'), ('la risposta', 42), ('la risposta', 'culo')]
Nodi correnti [7, '7', 42, 'la risposta', 'culo']
Lista di adiacenza {7: ['7'], '7': [], 42: [], 'la risposta': [42, 'culo'], 'culo': []}
Le risposte sono [42, 'culo']
Questo grafo è un Directed Graph
Ci sono 5 nodi e 3 lati
Il numero medio di link è 0.6
DiGraph()


Prova WeightedDiGraph()

In [100]:
prova = weightedDiGraph()
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_node(7)
prova.add_node('7')
prova.add_node(42)
prova.add_node('la risposta')
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_edge(7, '7', 1)
prova.add_edge(7, '7', 3)
prova.add_edge('la risposta', 42, 42)
prova.add_edge('la risposta', 'culo', 17)
print('Lati correnti', prova.edges)
print('Nodi correnti', prova.nodes)
print('Lista di adiacenza', prova.adj())
print('Le risposte sono', prova['la risposta'])
print('Questo grafo è un', prova.graph_type)
print('Ci sono', prova.n_nodes, 'nodi e', prova.n_edges, 'lati')
print('Il numero medio di link è', prova.get_average_links)
print(prova)

Lista di adiacenza {}
Nodi correnti []
Lista di adiacenza {7: {}, '7': {}, 42: {}, 'la risposta': {}}
Nodi correnti [7, '7', 42, 'la risposta']
Lati correnti [((7, '7'), 4), (('la risposta', 42), 42), (('la risposta', 'culo'), 17)]
Nodi correnti [7, '7', 42, 'la risposta', 'culo']
Lista di adiacenza {7: {'7': 4}, '7': {}, 42: {}, 'la risposta': {42: 42, 'culo': 17}, 'culo': {}}
Le risposte sono {42: 42, 'culo': 17}
Questo grafo è un Weighted Directed Graph
Ci sono 5 nodi e 3 lati
Il numero medio di link è 0.6
weightedDiGraph()


In [101]:
prova2 = weightedDiGraph()
prova2.add_node(84)
prova2.add_node('9')
prova2.add_node(173)
prova2.add_node('le risposte')
prova2.add_edge(84, '7', 1)
prova2.add_edge(7, '7', 92)
prova2.add_edge('le risposte', 42, 42)
prova2.add_edge('le risposte', 'culo', 17)
prova2.add_edge('culo', 'la risposta', 91)
print('Lati correnti', prova2.edges)
print('Nodi correnti', prova2.nodes)
print('Lista di adiacenza', prova2.adj())
print('Le risposte sono', prova2['la risposta'])
print('Questo grafo è un', prova2.graph_type)
print('Ci sono', prova2.n_nodes, 'nodi e', prova2.n_edges, 'lati')
print('Il numero medio di link è', prova2.get_average_links)
print(prova2)

Lati correnti [((84, '7'), 1), ((7, '7'), 92), (('le risposte', 42), 42), (('le risposte', 'culo'), 17), (('culo', 'la risposta'), 91)]
Nodi correnti [84, '9', 173, 'le risposte', '7', 7, 42, 'culo', 'la risposta']
Lista di adiacenza {84: {'7': 1}, '9': {}, 173: {}, 'le risposte': {42: 42, 'culo': 17}, '7': {}, 7: {'7': 92}, 42: {}, 'culo': {'la risposta': 91}, 'la risposta': {}}
Le risposte sono {}
Questo grafo è un Weighted Directed Graph
Ci sono 9 nodi e 5 lati
Il numero medio di link è 0.5555555555555556
weightedDiGraph()


In [102]:
prova2.union(prova)
print('Lati correnti', prova2.edges)
print('Nodi correnti', prova2.nodes)
print('Lista di adiacenza', prova2.adj())
print('Le risposte sono', prova2['la risposta'])
print('Questo grafo è un', prova2.graph_type)
print('Ci sono', prova2.n_nodes, 'nodi e', prova2.n_edges, 'lati')
print('Il numero medio di link è', prova2.get_average_links)
print(prova2)

Lati correnti [((84, '7'), 1), ((7, '7'), 96), (('le risposte', 42), 42), (('le risposte', 'culo'), 17), (('culo', 'la risposta'), 91), (('la risposta', 42), 42), (('la risposta', 'culo'), 17)]
Nodi correnti [84, '9', 173, 'le risposte', '7', 7, 42, 'culo', 'la risposta']
Lista di adiacenza {84: {'7': 1}, '9': {}, 173: {}, 'le risposte': {42: 42, 'culo': 17}, '7': {}, 7: {'7': 96}, 42: {}, 'culo': {'la risposta': 91}, 'la risposta': {42: 42, 'culo': 17}}
Le risposte sono {42: 42, 'culo': 17}
Questo grafo è un Weighted Directed Graph
Ci sono 9 nodi e 7 lati
Il numero medio di link è 0.7777777777777778
weightedDiGraph()


Prova MultiDiGraph()

In [103]:
prova = MultiDiGraph()
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_node(7)
prova.add_node('7')
prova.add_node(42)
prova.add_node('la risposta')
print('Lista di adiacenza', prova.adj())
print('Nodi correnti', prova.nodes)
prova.add_edge(7, '7', 7)
prova.add_edge('la risposta', 42, 42)
prova.add_edge('la risposta', 'culo', 45)
prova.add_edge(7, '7', 98)
print('Lati correnti', prova.edges)
print('Nodi correnti', prova.nodes)
print('Lista di adiacenza', prova.adj())
print('Le risposte sono', prova['la risposta'])
print('Questo grafo è un', prova.graph_type)
print('Ci sono', prova.n_nodes, 'nodi e', prova.n_edges, 'lati')
print('Il numero medio di link è', prova.get_average_links)
print(prova)

Lista di adiacenza {}
Nodi correnti []
Lista di adiacenza {7: [], '7': [], 42: [], 'la risposta': []}
Nodi correnti [7, '7', 42, 'la risposta']
Lati correnti [(7, '7', 7), ('la risposta', 42, 42), ('la risposta', 'culo', 45), (7, '7', 98)]
Nodi correnti [7, '7', 42, 'la risposta', 'culo']
Lista di adiacenza {7: [('7', 7), ('7', 98)], '7': [], 42: [], 'la risposta': [(42, 42), ('culo', 45)], 'culo': []}
Le risposte sono [(42, 42), ('culo', 45)]
Questo grafo è un Directed MultiGraph
Ci sono 5 nodi e 4 lati
Il numero medio di link è 0.8
DiMultiGraph()


In [107]:
wow = prova.get_DiGraph_from_interval(start_time = 41.1, end_time = 45, weight = 1)
print(wow.adj())
print(wow.edges)

{42: {}, 'la risposta': {42: 1, 'culo': 1}, 'culo': {}}
[(('la risposta', 42), 1), (('la risposta', 'culo'), 1)]


In [116]:
filename = 'data/sx-stackoverflow-a2q.txt'
prova_dataset = MultiDiGraph()
prova_dataset.get_graph(filename)
woah1 = prova_dataset.get_DiGraph_from_interval(start_time = 1200000000, end_time = 1300000000, weight = 1)

0it [00:00, ?it/s]

In [117]:
filename = 'data/sx-stackoverflow-c2a.txt'
prova_dataset = MultiDiGraph()
prova_dataset.get_graph(filename)
woah2 = prova_dataset.get_DiGraph_from_interval(start_time = 1200000000, end_time = 1300000000, weight = 0.5)

0it [00:00, ?it/s]

In [118]:
woah1.union(woah2)

In [ ]:
woah1.adj()